# Exercise 05


In [ ]:
%matplotlib qt5
import numpy as np
from scipy import sparse as sp
import lanczos
from matplotlib import pyplot as plt

In [ ]:
L = 14
g = 1.5
H = lanczos.gen_hamiltonian_L(L, g)
# psi_0 = np.random.choice([0, 1], size=H.shape[0], replace=True)
psi_0 = np.random.random(H.shape[0])
psi_0 /= np.linalg.norm(psi_0)

In [ ]:
# since g != 1, we do NOT expect ground state degeneracy
T, V = lanczos.lanczos(psi_0, H, N=200, stabilize=True)
T_unstable, V_unstable = lanczos.lanczos(psi_0, H, N=200, stabilize=False)

In [ ]:
# Check if V is orthonormal
# normal
assert np.allclose([np.linalg.norm(v) for v in V], 1)
# orthogonal
for i, Vi in enumerate(V):
    for j in range(i):
        assert abs(np.inner(Vi, V[j])) < 1e-13

In [ ]:
eigvals = np.linalg.eigvalsh(T)
unstable_eigvals = np.linalg.eigvalsh(T_unstable)
sp_eigvals, _ = sp.linalg.eigsh(T, k=10, which="SA")

In [ ]:
def eigenval_hist(vals: np.ndarray, method: str = ""):
    plt.figure()
    plt.hist(vals, bins=np.unique(vals).size)
    plt.xlabel("Eigenvalues")
    plt.ylabel("Count")
    plt.title(f"Eigenvalue degeneracy {method}")
    plt.show()
eigenval_hist(eigvals[:10], "lanczos")
eigenval_hist(unstable_eigvals[:10], "lanczos (unstable)")
eigenval_hist(sp_eigvals, "scipy")

As can be seen in the above plots, the "stabilize"-option removes the discrepancy between the scipy (arpack) implementation and the homemade lanczos implementation. With the "stabilize"-parameter set to "True", the algorithm ensures the vectors are orthogonal, whereas with the option disabled the vectors are only normalized. Hence, orthonormality is not fulfilled and ground state degeneracy might not be fulfilled either.